In [ ]:
# Steps:

## model
# drop the columns from the dataframe 
# scale
# we do the clustering
# plot 
# new column with the cluster and the id
# csv (to pass in the function)

## program creation
# 1 is it hot? no
# 2 search on spotify for the song
# 3 get the id
# 4 with the id get the json file with the features
# 5 scale/ transform
# 6 predict 
# 7 get the recommendation from the same cluster


In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import spotipy
import json
import config
import pprint
import textdistance
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt           

In [2]:
### ELSE PART:

import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

user_input = 'yesterday'

results = sp.search(q=user_input,limit=5,market="GB") ### to query the name of the song 
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=yesterday&type=track&market=GB&offset=0&limit=5',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'},
       'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2',
       'id': '3WrFJ7ztbogyGnTHbHJFl2',
       'name': 'The Beatles',
       'type': 'artist',
       'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/0PT5m6hwPRrpBwIHVnvbFX'},
     'href': 'https://api.spotify.com/v1/albums/0PT5m6hwPRrpBwIHVnvbFX',
     'id': '0PT5m6hwPRrpBwIHVnvbFX',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273e3e3b64cea45265469d4cafa',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02e3e3b64cea45265469d4cafa',
       'width': 300},
      {'height': 64,
       'url': 'https://i.

In [3]:
pprint.pprint(results['tracks']['items'][0]['id'])

'3BQHpFgAp4l80e1XslIjNI'


In [4]:
results["tracks"]["items"][0]["name"] # Track name

'Yesterday - Remastered 2009'

In [5]:
input_all_features = sp.audio_features(results["tracks"]["items"][0]["id"])[0]
input_all_features

{'danceability': 0.332,
 'energy': 0.179,
 'key': 5,
 'loudness': -11.83,
 'mode': 1,
 'speechiness': 0.0326,
 'acousticness': 0.879,
 'instrumentalness': 0,
 'liveness': 0.0886,
 'valence': 0.315,
 'tempo': 96.529,
 'type': 'audio_features',
 'id': '3BQHpFgAp4l80e1XslIjNI',
 'uri': 'spotify:track:3BQHpFgAp4l80e1XslIjNI',
 'track_href': 'https://api.spotify.com/v1/tracks/3BQHpFgAp4l80e1XslIjNI',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3BQHpFgAp4l80e1XslIjNI',
 'duration_ms': 125667,
 'time_signature': 4}

In [6]:
selected_features = ['danceability', 'energy','loudness', 'speechiness',
                    'acousticness', 'instrumentalness', 'liveness','tempo']

In [7]:
input_features = {feature: input_all_features[feature] for feature in selected_features}
input_features

{'danceability': 0.332,
 'energy': 0.179,
 'loudness': -11.83,
 'speechiness': 0.0326,
 'acousticness': 0.879,
 'instrumentalness': 0,
 'liveness': 0.0886,
 'tempo': 96.529}

In [8]:
input_df = pd.DataFrame([input_features])
input_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,tempo
0,0.332,0.179,-11.83,0.0326,0.879,0,0.0886,96.529


In [9]:
def load(filename="newscaler.pickle"):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        print("File not found!")

In [10]:
newscaler = load("newscaler.pickle")

In [11]:
scaled_features = newscaler.transform(input_df)
scaled_features

array([[-1.77194337, -2.79509648, -1.7887062 , -0.70289091,  3.36249192,
        -0.22396058, -0.66096447, -0.98105697]])

In [12]:
scaled_features_df = pd.DataFrame(scaled_features, columns = input_df.columns)
scaled_features_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,tempo
0,-1.771943,-2.795096,-1.788706,-0.702891,3.362492,-0.223961,-0.660964,-0.981057


In [13]:
# Loading a k-means model
kmeans = load(filename = "spotify_model.pickle")

In [14]:
predicted_cluster = kmeans.predict(scaled_features_df)[0]
predicted_cluster

16

In [16]:
X = pd.read_csv('new_df.csv')

In [34]:
new_recommended = X[X['cluster'] == predicted_cluster].sample()
new_recommended

,Unnamed: 0,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,tempo,cluster,id
5167,5167,0.684,0.574,-7.487,0.291,0.633,0.0,0.113,80.427,16,6FFKaCx6rav6iqj5ag8tTZ


In [41]:
new_recommended["id"].values[0]

'6FFKaCx6rav6iqj5ag8tTZ'

In [18]:
np.unique(kmeans.labels_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [21]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
                  width="320",
                  height="80",
                  frameborder="0",
                  allowtransparency="true",
                  allow="encrypted-media",
                  )


In [37]:
display(play_song("5565"))

In [24]:
kmeans = load(filename = "spotify_model.pickle")

In [27]:
np.unique(kmeans.labels_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])